In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import copy
import time
%matplotlib inline
np.random.seed(0)

In [4]:
from pymatch.Matcher import Matcher

In [12]:
drug1 = pd.read_csv('/Users/likehang/Desktop/data/drug1.csv')
drug2 = pd.read_csv('/Users/likehang/Desktop/data/drug2.csv')
drug3 = pd.read_csv('/Users/likehang/Desktop/data/drug3.csv')
drug4 = pd.read_csv('/Users/likehang/Desktop/data/drug4.csv')
drug5 = pd.read_csv('/Users/likehang/Desktop/data/drug5.csv')
drug6 = pd.read_csv('/Users/likehang/Desktop/data/drug6.csv')
drug7 = pd.read_csv('/Users/likehang/Desktop/data/drug7.csv')
drug8 = pd.read_csv('/Users/likehang/Desktop/data/drug8.csv')
drug9 = pd.read_csv('/Users/likehang/Desktop/data/drug9.csv')
drug10 = pd.read_csv('/Users/likehang/Desktop/data/drug10.csv')
d1 = pd.read_csv('/Users/likehang/Desktop/data/disease1.csv')
d2 = pd.read_csv('/Users/likehang/Desktop/data/disease2.csv')
d3 = pd.read_csv('/Users/likehang/Desktop/data/disease3.csv')
d4 = pd.read_csv('/Users/likehang/Desktop/data/disease4.csv')
d5 = pd.read_csv('/Users/likehang/Desktop/data/disease5.csv')
d6 = pd.read_csv('/Users/likehang/Desktop/data/disease6.csv')
d7 = pd.read_csv('/Users/likehang/Desktop/data/disease7.csv')
d8 = pd.read_csv('/Users/likehang/Desktop/data/disease8.csv')

In [13]:
drug1_p = set(drug1['patid'].values)
drug2_p = set(drug2['patid'].values)
drug3_p = set(drug3['patid'].values)
drug4_p = set(drug4['patid'].values)
drug5_p = set(drug5['patid'].values)
drug6_p = set(drug6['patid'].values)
drug7_p = set(drug7['patid'].values)
drug9_p = set(drug9['patid'].values)
drug10_p = set(drug10['patid'].values)
d2_p = set(d2['patid'].values)
d3_p = set(d3['patid'].values)
d4_p = set(d4['patid'].values)
d5_p = set(d5['patid'].values)
d6_p = set(d6['patid'].values)
d7_p = set(d7['patid'].values)
d8_p = set(d8['patid'].values)

In [14]:
meta_p = pd.concat([drug1,drug2,drug3,drug4,drug5,drug6,drug7,drug8,drug9,drug10,d1,d2,d3,d4,d5,d6,d7,d8], sort=True).drop_duplicates('patid')[['patid','GDR_CD','yrdob']]

In [15]:
last_date = max(d1['lst_dt'])
d1 = d1.drop_duplicates('patid', keep='first')
d2 = d2.drop_duplicates('patid', keep='first')
d3 = d3.drop_duplicates('patid', keep='first')
d4 = d4.drop_duplicates('patid', keep='first')
d5 = d5.drop_duplicates('patid', keep='first')
d6 = d6.drop_duplicates('patid', keep='first')
d7 = d7.drop_duplicates('patid', keep='first')
d8 = d8.drop_duplicates('patid', keep='first')
drug8 = drug8[drug8['VXX_dt']<=last_date-60]
drug8_p = set(drug8['patid'].values)
d1_p = set(d1['patid'].values)

In [16]:
patient = set(drug8['patid'].values)|\
          set(drug1['patid'].values)|\
          set(drug2['patid'].values)|\
          set(drug3['patid'].values)|\
          set(drug4['patid'].values)|\
          set(drug5['patid'].values)|\
          set(drug6['patid'].values)|\
          set(drug7['patid'].values)|\
          set(drug9['patid'].values)|\
          set(drug10['patid'].values)|\
          set(d2['patid'].values)|\
          set(d3['patid'].values)|\
          set(d4['patid'].values)|\
          set(d5['patid'].values)|\
          set(d6['patid'].values)|\
          set(d7['patid'].values)|\
          set(d8['patid'].values)-\
          set(d1.loc[d1['fst_dt']<60,'patid'].values)

In [17]:
patient_info = {'patid':[], 
       'date' : [],
       'd1_record': [],
       'drug8_record':[]
      }

date_min, date_max = min(drug8['VXX_dt']), max(drug8['VXX_dt'])

start_time = time.time()

for i in patient:
    patient_info['patid'].append(i)
    
    if i in drug8_p: # if take the drug 8
        patient_info['drug8_record'].append(1)
        
        if i in d1_p: # if have the disease 1
            diagnose =  d1.loc[d1['patid']==i,'fst_dt'].values[0]
            sub = drug8[(drug8['patid']==i) & (drug8['VXX_dt']<diagnose-14) &  (drug8['sup_dt']>diagnose-60)]
            if len(sub) == 0: # if no record satisfies the time condition
                patient_info['date'].append(np.random.choice(drug8[drug8['patid']==i]['VXX_dt'].values))
                patient_info['d1_record'].append(0) # randomly pick one record
                #patient_info['patid'].pop()
            else: # if at least one record satisfies the time condition
                patient_info['date'].append(min(sub['VXX_dt'].values))
                patient_info['d1_record'].append(1) # pick the first record
                #patient_info['drug8_record'].append(1)
                
        else: # if no disease 1
            patient_info['date'].append(np.random.choice(drug8[drug8['patid']==i]['VXX_dt'].values))
            patient_info['d1_record'].append(0) # randomly pick one record
            #patient_info['drug8_record'].append(1)
        
    else: # if not take the drug 8
        patient_info['drug8_record'].append(0)
        
        if i in d1_p: # if have the disease 1
            patient_info['date'].append(d1.loc[d1['patid']==i, 'fst_dt'].values[0] - np.random.randint(14,60))
            patient_info['d1_record'].append(1) # randomly pick a date 14-60 days before getting the disease
            
        else: # if no disease 1
            patient_info['date'].append(np.random.randint(date_min, date_max))
            patient_info['d1_record'].append(0) # randomly pick a date between first and last date of taking drug 8

print("For loop old takes {}".format(time.time()-start_time))

patient_info = pd.DataFrame(patient_info)    

For loop old takes 451.5774872303009


In [18]:
patient_info_tmp = pd.merge(patient_info, drug3, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug3']=1 #If taking drug3 before the date, set column ‘drug3’ to be 1
patient_info_tmp.loc[patient_info_tmp['drug3']!=1,'drug3']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug3']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug3'] == 1]
patient_info_tmp_date.loc[:,'drug3_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug3_date']<0,'drug3_date'] = 0 #exacting all record with drug3 = 1, calculate the latest time taking drug 3
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug3_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left') #merge

patient_info_tmp = pd.merge(patient_info, drug10, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug10']=1
patient_info_tmp.loc[patient_info_tmp['drug10']!=1,'drug10']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug10']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug10'] == 1]
patient_info_tmp_date.loc[:,'drug10_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug10_date']<0,'drug10_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug10_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug8, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug8']=1
patient_info_tmp.loc[patient_info_tmp['drug8']!=1,'drug8']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug8']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug8'] == 1]
patient_info_tmp_date.loc[:,'drug8_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug8_date']<0,'drug8_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug8_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug1, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug1']=1
patient_info_tmp.loc[patient_info_tmp['drug1']!=1,'drug1']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug1']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug1'] == 1]
patient_info_tmp_date.loc[:,'drug1_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug1_date']<0,'drug1_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug1_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug2, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug2']=1
patient_info_tmp.loc[patient_info_tmp['drug2']!=1,'drug2']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug2']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug2'] == 1]
patient_info_tmp_date.loc[:,'drug2_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug2_date']<0,'drug2_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug2_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug4, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug4']=1
patient_info_tmp.loc[patient_info_tmp['drug4']!=1,'drug4']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug4']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug4'] == 1]
patient_info_tmp_date.loc[:,'drug4_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug4_date']<0,'drug4_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug4_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug5, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug5']=1
patient_info_tmp.loc[patient_info_tmp['drug5']!=1,'drug5']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug5']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug5'] == 1]
patient_info_tmp_date.loc[:,'drug5_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug5_date']<0,'drug5_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug5_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug6, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug6']=1
patient_info_tmp.loc[patient_info_tmp['drug6']!=1,'drug6']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug6']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug6'] == 1]
patient_info_tmp_date.loc[:,'drug6_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug6_date']<0,'drug6_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug6_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug7, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug7']=1
patient_info_tmp.loc[patient_info_tmp['drug7']!=1,'drug7']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug7']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug7'] == 1]
patient_info_tmp_date.loc[:,'drug7_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug7_date']<0,'drug7_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug7_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, drug9, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['VXX_dt']<patient_info_tmp['date'],'drug9']=1
patient_info_tmp.loc[patient_info_tmp['drug9']!=1,'drug9']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','drug9']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['drug9'] == 1]
patient_info_tmp_date.loc[:,'drug9_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['sup_dt'] 
patient_info_tmp_date.loc[patient_info_tmp_date['drug9_date']<0,'drug9_date'] = 0
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','drug9_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d3, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d3']=1
patient_info_tmp.loc[patient_info_tmp['d3']!=1,'d3']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','d3']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d3'] == 1]
patient_info_tmp_date.loc[:,'d3_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d3_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d4, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d4']=1
patient_info_tmp.loc[patient_info_tmp['d4']!=1,'d4']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','d4']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d4'] == 1]
patient_info_tmp_date.loc[:,'d4_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d4_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d6, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d6']=1
patient_info_tmp.loc[patient_info_tmp['d6']!=1,'d6']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','d6']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d6'] == 1]
patient_info_tmp_date.loc[:,'d6_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d6_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d7, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d7']=1
patient_info_tmp.loc[patient_info_tmp['d7']!=1,'d7']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','d7']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d7'] == 1]
patient_info_tmp_date.loc[:,'d7_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d7_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d8, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d8']=1
patient_info_tmp.loc[patient_info_tmp['d8']!=1,'d8']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','d8']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d8'] == 1]
patient_info_tmp_date.loc[:,'d8_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d8_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d2, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d2']=1
patient_info_tmp.loc[patient_info_tmp['d2']!=1,'d2']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','d2']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d2'] == 1]
patient_info_tmp_date.loc[:,'d2_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d2_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info_tmp = pd.merge(patient_info, d5, on = 'patid', how = 'left')
patient_info_tmp.loc[patient_info_tmp['fst_dt']<patient_info_tmp['date'],'d5']=1
patient_info_tmp.loc[patient_info_tmp['d5']!=1,'d5']=0
patient_info_tmp_10 = patient_info_tmp.drop_duplicates('patid')
patient_info_tmp_10 = patient_info_tmp_10[['patid','d5']]
patient_info_tmp_date = patient_info_tmp[patient_info_tmp['d5'] == 1]
patient_info_tmp_date.loc[:,'d5_date'] = patient_info_tmp_date['date'] - patient_info_tmp_date['fst_dt']
patient_info_tmp_date = patient_info_tmp_date.drop_duplicates('patid', keep='last')
patient_info_tmp_date = patient_info_tmp_date[['patid','d5_date']]
patient_info = pd.merge(patient_info, patient_info_tmp_10, on='patid', how ='left')
patient_info = pd.merge(patient_info, patient_info_tmp_date, on = 'patid', how = 'left')

patient_info = pd.merge(patient_info, meta_p, on='patid', how='left')
patient_info.to_csv('/Users/likehang/Desktop/data/patient_info.csv')

In [9]:
patient_info_8 = patient_info[patient_info['drug8_record']==1]
patient_info_no_8 = patient_info[patient_info['drug8_record']==0]

In [10]:
len(patient_info_8[patient_info_8['d1_record']==1]), len(patient_info_8[patient_info_8['d1_record']==0]), len(patient_info_no_8[patient_info_no_8['d1_record']==1]), len(patient_info_no_8[patient_info_no_8['d1_record']==0])

(127, 233410, 36626, 9604445)

In [11]:
d = np.array([[127, 233410], [36626, 9604445]])
chi2_contingency(d)

(650.6730860039967,
 1.5948812180801366e-143,
 1,
 array([[8.69217832e+02, 2.32667782e+05],
        [3.58837822e+04, 9.60518722e+06]]))

In [13]:
patient_info_8.index = np.arange(len(patient_info_8))
patient_info_no_8.index = np.arange(len(patient_info_no_8))
no_index = []
yes_index = []

start_time = time.time()

dic_no_8_proxy = {}
for i in range(len(patient_info_no_8)):
    v1 = str(patient_info_no_8.loc[i, 'drug1'])
    v2 = str(patient_info_no_8.loc[i, 'drug2'])
    v3 = str(patient_info_no_8.loc[i, 'drug3'])
    v4 = str(patient_info_no_8.loc[i, 'drug4'])
    v5 = str(patient_info_no_8.loc[i, 'drug5'])
    v6 = str(patient_info_no_8.loc[i, 'drug6'])
    v7 = str(patient_info_no_8.loc[i, 'drug7'])
    v8 = str(patient_info_no_8.loc[i, 'drug8'])
    v9 = str(patient_info_no_8.loc[i, 'drug9'])
    v10 = str(patient_info_no_8.loc[i, 'drug10'])
    v11 = str(patient_info_no_8.loc[i, 'd2'])
    v12 = str(patient_info_no_8.loc[i, 'd3'])
    v13 = str(patient_info_no_8.loc[i, 'd4'])
    v14 = str(patient_info_no_8.loc[i, 'd5'])
    v15 = str(patient_info_no_8.loc[i, 'd6'])
    v16 = str(patient_info_no_8.loc[i, 'd7'])
    v17 = str(patient_info_no_8.loc[i, 'd8'])
    v18 = str(patient_info_no_8.loc[i, 'GDR_CD'])
    v19 = str(patient_info_no_8.loc[i, 'yrdob'])
    v_proxy = ''.join([v1, v2, v3, v4, v5, v6, v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19]) # yrdob can be any value
    if v_proxy not in dic_no_8_proxy:
        dic_no_8_proxy[v_proxy] = [i]
    else:
        dic_no_8_proxy[v_proxy].append(i)
        
for i in range(len(patient_info_8)):
    v1 = str(patient_info_8.loc[i, 'drug1'])
    v2 = str(patient_info_8.loc[i, 'drug2'])
    v3 = str(patient_info_8.loc[i, 'drug3'])
    v4 = str(patient_info_8.loc[i, 'drug4'])
    v5 = str(patient_info_8.loc[i, 'drug5'])
    v6 = str(patient_info_8.loc[i, 'drug6'])
    v7 = str(patient_info_8.loc[i, 'drug7'])
    v8 = str(patient_info_8.loc[i, 'drug8'])
    v9 = str(patient_info_8.loc[i, 'drug9'])
    v10 = str(patient_info_8.loc[i, 'drug10'])
    v11 = str(patient_info_8.loc[i, 'd2'])
    v12 = str(patient_info_8.loc[i, 'd3'])
    v13 = str(patient_info_8.loc[i, 'd4'])
    v14 = str(patient_info_8.loc[i, 'd5'])
    v15 = str(patient_info_8.loc[i, 'd6'])
    v16 = str(patient_info_8.loc[i, 'd7'])
    v17 = str(patient_info_8.loc[i, 'd8'])
    v18 = str(patient_info_8.loc[i, 'GDR_CD'])
    v19 = str(patient_info_8.loc[i, 'yrdob'])
    v_proxy = ''.join([v1, v2, v3, v4, v5, v6, v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19])
    if v_proxy in dic_no_8_proxy:
        yes_index.append(i)
        tmp = np.random.choice(dic_no_8_proxy[v_proxy])
        no_index.append(tmp) # do not drop (bootstrap)
        
print("For loops take {}".format(time.time()-start_time))

no_drug = patient_info_no_8.loc[no_index,:]
yes_drug = patient_info_8.loc[yes_index,:]

For loops take 4568.314674854279


In [17]:
len(no_drug) # the reason that the records are more is that Shizhang deletes each record in patient_info_no_8 after matching it
             # with a record in patient_info_8 so some of the records in patient_info_8 may not have matching record just because it
             # appears late in the data set, while I randomly select the matching records in patient_info_no_8 for each
             # record in patient_info_8.

183635

In [18]:
len(no_drug[no_drug['d1_record'] == 1])

793

In [19]:
len(no_drug[no_drug['d1_record'] == 0])

182842

In [20]:
len(yes_drug[yes_drug['d1_record'] == 1])

67

In [21]:
len(yes_drug[yes_drug['d1_record'] == 0])

183568

In [22]:
# Shizhang
d = np.array([[793, 182842], [29, 183568]])
chi2_contingency(d)

(709.6653955629511,
 2.365488123746888e-156,
 1,
 array([[   411.04252897, 183223.95747103],
        [   410.95747103, 183186.04252897]]))